In [1]:
import cv2
import numpy as np
import math
from random import randrange
import time
import msvcrt as m
import pyttsx3

cap = cv2.VideoCapture(0)
converter = pyttsx3.init()
converter.setProperty('rate', 150)
converter.setProperty('volume', 100)
voice_id = "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0"
converter.setProperty('voice', voice_id)

    
def hint(correct_ans, your_ans):
    if correct_ans > your_ans:
        to_add = int(correct_ans) - int(your_ans)
        converter = pyttsx3.init()
        converter.setProperty('rate', 150)
        converter.setProperty('volume', 100)
        voice_id = "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0"
        converter.setProperty('voice', voice_id)
        converter.say("Your answer is " + str(to_add) + " less than the correct answer")
        converter.runAndWait()
    elif correct_ans < your_ans:
        to_subtract = (int(correct_ans) - int(your_ans)) * -1
        converter = pyttsx3.init()
        converter.setProperty('rate', 150)
        converter.setProperty('volume', 100)
        voice_id = "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0"
        converter.setProperty('voice', voice_id)
        converter.say("Your answer is " + str(to_subtract) + " more than the correct answer")
        converter.runAndWait()
    else:
        pass

number = str(randrange(6))
converter.say("Hi, let's learn to count!!")
converter.runAndWait()
converter.say("Show number" + number + "using your hand")
converter.runAndWait()
points = 0
go_on = 0
while(1):
    cont = 0
    try:  #an error comes if it does not find anything in window as it cannot find contour of max area
          #therefore this try error statement
        
        ret, frame = cap.read()
        frame=cv2.flip(frame,1)
        kernel = np.ones((3,3),np.uint8)
        
        #define region of interest
        roi=frame[10:300, 10:300]
        
        
        cv2.rectangle(frame,(100,100),(300,300),(0,255,0),0)    
        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        
        
         
    # define range of skin color in HSV
        lower_skin = np.array([0,20,70], dtype=np.uint8)
        upper_skin = np.array([20,255,255], dtype=np.uint8)
        
     #extract skin colur imagw  
        mask = cv2.inRange(hsv, lower_skin, upper_skin)
        
   
        
    #extrapolate the hand to fill dark spots within
        mask = cv2.dilate(mask,kernel,iterations = 4)
        
    #blur the image
        mask = cv2.GaussianBlur(mask,(5,5),100) 
        
        
    #find contours
        contours,hierarchy = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
   #find contour of max area(hand)
        cnt = max(contours, key = lambda x: cv2.contourArea(x))
        
    #approx the contour a little
        epsilon = 0.0005*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True)
       
        
    #make convex hull around hand
        hull = cv2.convexHull(cnt)
        
     #define area of hull and area of hand
        areahull = cv2.contourArea(hull)
        areacnt = cv2.contourArea(cnt)
      
    #find the percentage of area not covered by hand in convex hull
        arearatio=((areahull-areacnt)/areacnt)*100
    
     #find the defects in convex hull with respect to hand
        hull = cv2.convexHull(approx, returnPoints=False)
        defects = cv2.convexityDefects(approx, hull)
        
    # l = no. of defects
        l=0
        
    #code for finding no. of defects due to fingers
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(approx[s][0])
            end = tuple(approx[e][0])
            far = tuple(approx[f][0])
            pt= (100,180)
            
            
            # find length of all sides of triangle
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            s = (a+b+c)/2
            ar = math.sqrt(s*(s-a)*(s-b)*(s-c))
            
            #distance between point and convex hull
            d=(2*ar)/a
            
            # apply cosine rule here
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
            
        
            # ignore angles > 90 and ignore points very close to convex hull(they generally come due to noise)
            if angle <= 90 and d>30:
                l += 1
                cv2.circle(roi, far, 3, [255,0,0], -1)
            
            #draw lines around hand
            cv2.line(roi,start, end, [0,255,0], 2)
            
            
        l+=1
        
        #print corresponding gestures which are in their ranges
        font = cv2.FONT_HERSHEY_SIMPLEX
        if l==1:
            if areacnt<2000:
                cv2.putText(frame,'Put hand in the box',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            else:
                if arearatio<15:
                    cv2.putText(frame,'0',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    if int(number) == 0:
                        points += 10
                        number = str(randrange(6))
                        converter.say("Correct!!, this is zero")
                        converter.say("That's great, you now have" + str(points) + "points!")
                        if points != 100:
                            converter.say("Show number" + number + "using your hand")
                            converter.runAndWait()
                    hint(int(number), 0)
                        
                #elif arearatio<17.5:
                    #cv2.putText(frame,'Best of luck',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    #go_on = 1
                
                else:
                    cv2.putText(frame,'1',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    if int(number) == 1:
                        points += 10
                        number = str(randrange(6))
                        converter.say("Correct!!, this is one")
                        converter.say("That's great, you now have" + str(points) + "points!")
                        if points != 100:
                            converter.say("Show number" + number + "using your hand")
                            converter.runAndWait()
                    hint(int(number), 1)
        elif l==2:
            cv2.putText(frame,'2',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            if int(number) == 2:
                points += 10
                number = str(randrange(6))
                converter.say("Correct!!, this is two")
                converter.say("That's great, you now have" + str(points) + "points!")
                if points != 100:
                    converter.say("Show number" + number + "using your hand")
                    converter.runAndWait()
            hint(int(number), 2)
                
        elif l==3:
         
              if arearatio<32:
                    cv2.putText(frame,'3',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    if int(number) == 3:
                        points += 10
                        number = str(randrange(6))
                        converter.say("Correct!!, this is 3")
                        converter.say("That's great, you now have" + str(points) + "points!")
                        if points != 100:
                            converter.say("Show number" + number + "using your hand")
                            converter.runAndWait()
                    hint(int(number), 3)
              
        elif l==4:
            cv2.putText(frame,'4',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            if int(number) == 4:
                points += 10
                number = str(randrange(6))
                converter.say("Correct!!, this is 4")
                converter.say("That's great, you now have" + str(points) + "points!")
                if points != 100:
                    converter.say("Show number" + number + "using your hand")
                    converter.runAndWait()
            hint(int(number), 4)
            
        elif l==5:
            cv2.putText(frame,'5',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            if int(number) == 5:
                points += 10
                number = str(randrange(6))
                converter.say("Correct!!, this is 5")
                converter.say("That's great, you now have" + str(points) + "points!")
                if points != 100:
                    converter.say("Show number" + number + "using your hand")
                    converter.runAndWait()
            hint(int(number), 5)
                
        elif l==6:
            cv2.putText(frame,'Reposition',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        else :
            cv2.putText(frame,'Reposition',(10,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        #show the windows
        cv2.imshow('mask',mask)
        cv2.imshow('frame',frame)
    except:
        pass
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
    if int(points) == 100:
        converter.say("Congratulations, you have reached the daily goal of 100 points, keep up the good work, see you soon!")
        converter.runAndWait()
        break
cv2.destroyAllWindows()
cap.release()